*Okay. Alright. That's fine.*

\- Drake



Hyperparameter optimization did not as expected, but we have more exciting things ahead of us: creating our own graph classifier. Let's approach this task using PyTorch Geometric using this [example](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=mHSP6-RBOqCE) as reference.

We can break this down into a few steps:
- Convert SMILES data into graph data
- Mini-batch graph data
- Define Graph Neural Network for graph classification
- Train model and evaluate - you know the drill 

Let's get to it 🤖

In [ ]:
!pip install deepchem
!pip install 'deepchem[torch]'
!pip install rdkit
!pip install torch_geometric

In [2]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv', reload=False)
train_dataset, valid_dataset, test_dataset = datasets

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading some Jax models, missing a dependency. No module named 'jax'
[11:43:08] WARNING: not removing hydrogen atom without neighbors
[11:43:20] WARNING: not removing hydrogen atom without neighbors


Let's take a close look at this dataset by inspecting the training dataset.

In [3]:
print(train_dataset)
print(train_dataset.X)

<DiskDataset X.shape: (6264,), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>
 <deepchem.feat.mol_graphs.ConvMol object at 0x2cf06d3c0> ...


It looks like there are **6264** graphs (molecules) in our training dataset, where *X* stores the molecules as ConvMol objects, *y* stores the hot-encoded output vector with one entry for each of the **12** measures/tasks, and a weights matrix *w*.

Here's one particular molecule:

In [4]:
test_mol = train_dataset.X[0]
print(f"This molecule has {test_mol.n_atoms} atoms with {test_mol.n_feat} features each.")

This molecule has 11 atoms with 75 features each.


In [5]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(6264, 783, 784)

Note above sizes of our training, validation, and test datasets. However, we can't just operate on the above data directly as molecular information is stored as a [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string (that is, simplified molecular-input line-entry system). Try saying that five times fast. In short, the specification enables structural information of molecules to be encoded into a string.

For example,

In [6]:
train_dataset.ids[0]

'CC(O)(P(=O)(O)O)P(=O)(O)O'

Fortunately, open-source is once again our savior: [SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spmatrix.html) supplies a few helper functions to convert sparse adjacency matrices  into graph data that we *can* use with PyTorch Geometric. From there, we can extract additional metadata to help understand the numbers.

In [7]:
import scipy.sparse
import numpy as np
def adjacency_list_to_sparse(adj_list):
    num_nodes = len(adj_list)
    rows, cols = [], []

    for i, neighbors in enumerate(adj_list):
        rows.extend([i] * len(neighbors))
        cols.extend(neighbors)

    adjacency_matrix = scipy.sparse.coo_matrix((np.ones_like(rows), (rows, cols)),
                                              shape=(num_nodes, num_nodes),
                                              dtype=np.float32)

    return adjacency_matrix


In [8]:
import torch
from torch_geometric.data import Data
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.utils.convert import from_scipy_sparse_matrix


def to_data(mol_graph):

    x = mol_graph.get_atom_features()

    adj_list = mol_graph.get_adjacency_list()
    sparse_mat = adjacency_list_to_sparse(adj_list)
    edge_index, edge_attr = from_scipy_sparse_matrix(sparse_mat)
    
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

A little clarification on the `Data` object might help:

1. **x (Node Feature Matrix):**
   - This parameter represents the feature matrix for each node in the graph.
   - It is a PyTorch tensor with shape [num_nodes, num_node_features].
   - Each row corresponds to a node, and each column corresponds to a feature of that node.
   - For example, if you are representing atoms in a molecule, `x` could contain features like atomic number, charge, etc.


2. **edge_index (Graph Connectivity):**
   - `edge_index` represents the graph connectivity in COO (Coordinate List) format.
   - It is a PyTorch tensor with shape [2, num_edges].
   - Each column of `edge_index` contains the indices of two nodes that form an edge.
   - For an undirected graph, (i, j) and (j, i) should both be present in the `edge_index`.


3. **edge_attr (Edge Feature Matrix):**
   - `edge_attr` represents the feature matrix for each edge in the graph.
   - It is a PyTorch tensor with shape [num_edges, num_edge_features].
   - Each row corresponds to an edge, and each column corresponds to a feature of that edge.
   - This is often used to store information like bond types, distances, or any other edge-specific features.

While there are a few other parameters, these three collectively provide a comprehensive representation of the graph needed for our specific use case.

Let's convert all our datasets from ConvMol to Data objects.

In [9]:
train_dataset_graph = []
for mol_graph in train_dataset.X:
    data = to_data(mol_graph)
    train_dataset_graph.append(data)

valid_dataset_graph = []
for mol_graph in valid_dataset.X:
    data = to_data(mol_graph)
    valid_dataset_graph.append(data)

test_dataset_graph = []
for mol_graph in test_dataset.X:
    data = to_data(mol_graph)
    test_dataset_graph.append(data)

In [10]:
train_dataset_graph[0]

Data(x=[11, 75], edge_index=[2, 20], edge_attr=[20])

In [11]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_dataset_graph, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset_graph, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [12]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 1942], edge_attr=[1942], batch=[960], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2308], edge_attr=[2308], batch=[1127], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 1990], edge_attr=[1990], batch=[978], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2006], edge_attr=[2006], batch=[993], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2062], edge_attr=[2062], batch=[1025], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2102], edge_attr=[2102], batch=[1037], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge_index=[2, 2316], edge_attr=[2316], batch=[1120], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(x=[64], edge

Hmm. Slight issue with the x parameter, but no clear solution is popping out to me at the moment. As my friend Dan likes to say, "Let's circle back to this."

Also, small (*very* consequential) update: after spending 2 days trying to get perfectly preprocess this data, I have discovered that PyTorch Geometric supplies its own MoleculeNet class, complete with a Tox 21 dataset 🥲.

Despite how much my RAM has suffered due to tabs upon tabs of documentation, I'd say that this process helped clarify **how** and **why** we preprocess our data. Now let's condense yesterday's work into three lines!

In [13]:
from torch_geometric.datasets import MoleculeNet
dataset = MoleculeNet(root="./data/", name="Tox21")
dataset.data

/Users/mihirarya/Dev/bcil/tox21/env/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[145459, 9], edge_index=[2, 302190], edge_attr=[302190, 3], smiles=[7831], y=[7831, 12])

In [14]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Tox21(7831):
Number of graphs: 7831
Number of features: 9
Number of classes: 12

Data(x=[16, 9], edge_index=[2, 34], edge_attr=[34, 3], smiles='CCOc1ccc2nc(S(N)(=O)=O)sc2c1', y=[1, 12])
Number of nodes: 16
Number of edges: 34
Average node degree: 2.12
Has isolated nodes: False
Has self-loops: False
Is undirected: True


There are 12 targets to predict, which is 11 too many to do at once - let's select NR-AhR (column 3) as the target to predict. Some of the rows in our dataset have a `NaN` value for this task, so we must them filter out:

In [15]:
dataset[0].y[0]

tensor([0., 0., 1., nan, nan, 0., 0., 1., 0., 0., 0., 0.])

Some datatype manipulation so we don't run into issues downstream!

In [16]:
dataset.y = dataset.y.long()

In [17]:
dataset.shuffle()
column_index_to_check = 2
filtered_data_list = [data for data in dataset if not torch.isnan(data.y[0, column_index_to_check]).any()]

In [18]:
split = int(len(filtered_data_list) * 0.80)
train_dataset, test_dataset = filtered_data_list[:split], filtered_data_list[split:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 5239
Number of test graphs: 1310


Now we're cooking with this dataset: shuffled, filtered, and split. We can finally move on...

Next step: **Mini-batching**.

Instead of "stacking" equally-sized matrices into a single mini-batch, as we may have done with image data, we take an alternative approach with graph data. "Why overcomplicate things?" you may ask. According to PyTorch Geometric [documentation](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=0gZ-l0npPIca):

1. GNN operators that rely on a **message passing scheme** (more on this later) do not need to be modified since messages are not exchanged between two nodes that belong to different graphs

2. There is no computational or memory overhead since adjacency matrices are saved in a sparse fashion holding only non-zero entries (*i.e.*, the edges)

PyTorch Geometric automatically takes care of **batching multiple graphs into a single giant graph** with the help of the [`torch_geometric.data.DataLoader`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.DataLoader) class:

In [19]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[1284, 9], edge_index=[2, 2656], edge_attr=[2656, 3], smiles=[64], y=[64, 12], batch=[1284], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[1139, 9], edge_index=[2, 2374], edge_attr=[2374, 3], smiles=[64], y=[64, 12], batch=[1139], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[1266, 9], edge_index=[2, 2584], edge_attr=[2584, 3], smiles=[64], y=[64, 12], batch=[1266], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[1130, 9], edge_index=[2, 2330], edge_attr=[2330, 3], smiles=[64], y=[64, 12], batch=[1130], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[1109, 9], edge_index=[2, 2264], edge_attr=[2264, 3], smiles=[64], y=[64, 12], batch=[1109], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[1095, 9], edge_index=[2, 2272], edge_attr=[2272, 3], smiles=[64], y=[64, 12], batch=[1095], ptr=[65])

Step

Mini-batch data ✅

Next step: Graph. Neural. Networks.

FINALLY. Let's make like Merriam-Webster and define what our GNN is going to look like.

But before we that, let's quickly review message-passing (you're welcome, future me).

!TODO: EXPLAIN MESSAGE PASSING

Now for the main course: defining the model. Thank you to the wonderful people at [PyTorch](https://www.learnpytorch.io/02_pytorch_classification/) and [PyTorch Geometric](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=mHSP6-RBOqCE) for these thoroughly-documented references (they low-key carried). Take a look at them if you haven't already!

In [20]:
from torch_geometric.nn import GraphConv
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1) # Output a binary value (0 or 1) because this is a binary classification problem 

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)

        x = x.to(self.lin.weight.dtype)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)

GNN(
  (conv1): GraphConv(9, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=1, bias=True)
)


Define Graph Neural Network for graph classification ✅


And finally, the code that we'll use to train and test our model.

In [21]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()


def train(loader):
    model.train()

    train_loss, train_acc = 0, 0
    for data in loader:  # Iterate in batches over the training dataset.
         
         # 1. Forward pass (model outputs raw logits)
         y_logits = model(data.x.to(torch.float32), data.edge_index, data.batch)
         y_preds = torch.round(torch.sigmoid(y_logits))
         y_train = data.y[:, column_index_to_check] # extract target column
         
         # 2. Calculate loss/accuracy
         loss = loss_fn(y_logits.squeeze(), y_train)
         train_loss += loss
         train_acc += accuracy_fn(y_true=y_train, y_pred=y_preds.squeeze())

         # 3. Optimizer zero grad
         optimizer.zero_grad() 

         # 4. Loss backwards
         loss.backward()  

         # 5. Optimizer step
         optimizer.step()
    train_loss /= len(loader)
    train_acc /= len(loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test(loader):
     model.eval()

     test_loss, test_acc = 0, 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         test_logits = model(data.x.to(torch.float32), data.edge_index, data.batch)
         test_preds = torch.round(torch.sigmoid(test_logits))
         labels = data.y[:, column_index_to_check] # extract target column

         test_loss += loss_fn(test_logits.squeeze(), labels)
         test_acc += accuracy_fn(y_true=labels, y_pred=test_preds.squeeze())
     test_loss /= len(loader)
     test_acc /= len(loader)
     print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")



In [23]:
for epoch in range(1, 100):
    train(train_loader)
    test(test_loader)

Train loss: 0.46709 | Train accuracy: 86.55%
Test loss: 0.30009 | Test accuracy: 88.61%

Train loss: 0.32101 | Train accuracy: 88.04%
Test loss: 0.29506 | Test accuracy: 88.61%

Train loss: 0.30924 | Train accuracy: 88.25%
Test loss: 0.31156 | Test accuracy: 88.61%

Train loss: 0.30842 | Train accuracy: 88.24%
Test loss: 0.29471 | Test accuracy: 88.61%

Train loss: 0.30941 | Train accuracy: 88.25%
Test loss: 0.29283 | Test accuracy: 88.61%

Train loss: 0.30037 | Train accuracy: 88.25%
Test loss: 0.28817 | Test accuracy: 88.61%

Train loss: 0.30182 | Train accuracy: 88.24%
Test loss: 0.29172 | Test accuracy: 88.61%

Train loss: 0.30725 | Train accuracy: 88.24%
Test loss: 0.28652 | Test accuracy: 88.61%

Train loss: 0.29302 | Train accuracy: 88.34%
Test loss: 0.28141 | Test accuracy: 89.28%

Train loss: 0.29313 | Train accuracy: 88.46%
Test loss: 0.28418 | Test accuracy: 89.26%

Train loss: 0.29138 | Train accuracy: 88.13%
Test loss: 0.28410 | Test accuracy: 88.61%

Train loss: 0.29365 |

Train model and evaluate ✅

We did it!